In [ ]:
#Librairies nécessaires
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
#Charge le mois en cours

#############################################################
#############################################################
mois = pd.read_excel("../2025/T1/Requete PC_AC_2025_01.xlsx", sheet_name=0) #Il faudra changer le chemin et/ou le nom du fichier ici d'un mois/trimestre/année à l'autre
#############################################################
#############################################################

mois

In [ ]:
#Charge combi.xlsx
combi363 = pd.read_excel('../Extractions/combi 363.xlsx', sheet_name=0)
combi363.rename(columns={'DCS_SEQ':'Numéro de séquence'}, inplace=True)
combi363['Combi'] = 363
combi363

In [ ]:
#Charge delegation.xlsx
delegation = pd.read_excel('../Extractions/delegation.xlsx', sheet_name=0)
delegation = delegation[:-1]
delegation['DCS_SEQ'] = delegation['DCS_SEQ'].astype(int)
delegation.rename(columns={'DCS_SEQ':'Numéro de séquence', 'DLG_COD':'Délégation'}, inplace=True)
delegation

In [ ]:
#Charge regul.xlsx et franchise.xlsx
#Elles serviront à créer la colonne "Type de remboursement"
regul = pd.read_excel('../Extractions/regul.xlsx', sheet_name=0)
regul.rename(columns={'DCS_SEQ':'Numéro de séquence'}, inplace=True)
regul['Flag regul'] = 'O'
display(regul)
franchise = pd.read_excel('../Extractions/franchise.xlsx', sheet_name=0)
franchise.rename(columns={'DCS_SEQ':'Numéro de séquence'}, inplace=True)
franchise['Flag franchise'] = 'O'
display(franchise)

In [ ]:
#Charge zpy.csv
zpy = pd.read_csv('../Extractions/zpy.csv')
zpy.rename(columns={'PAY_NOM':"Pays d'exécution", 'ZPY_COD':'Code région'}, inplace=True)
zpy

In [ ]:
#On effectue les jointures
test = (
    mois
    .merge(combi363, on='Numéro de séquence', how='left')
    .merge(delegation, on='Numéro de séquence', how='left')
    .merge(regul, on='Numéro de séquence', how='left')
    .merge(franchise, on ='Numéro de séquence', how='left')
    .merge(zpy, on="Pays d'exécution", how='left')
)

test

In [ ]:
#On vérifie pour quels pays le Code région est vide
test.loc[test['Code région'].isna()]["Pays d'exécution"].unique()

In [ ]:
#Il ne s'agit que de la France donc on remplace les valeurs manquantes par 0
test['Code région'] = test['Code région'].fillna(0).astype(int)
test

In [ ]:
#Liste des franchises actes
franchises_actes = ['FRAM', 'FRPH', 'FRTR', 'FDAM', 'FDPH', 'FDTR']

#On ajoute la colonne vide "Type de remboursement" en avant-dernière position
test.insert(len(test.columns)-1, 'Type de remboursement', None)

#Fonction pour appliquer la logique dans la nouvelle colonne
def assign_type(row):
    if row['Flag regul'] == 'O' and row['Code acte'] not in franchises_actes:
        return 'Régularisation'
    elif row['Flag franchise'] == 'O' and row['Code acte'] in franchises_actes:
        return 'Franchises'
    elif pd.isna(row['Type de remboursement']) and row['Code acte'] not in franchises_actes:
        return 'Remboursement'
    else:
        return None  #Si aucune condition n'est remplie, la cellule reste vide
    
#On applique puis on retire les colonnes Flag qui sont désormais inutiles
test['Type de remboursement'] = test.apply(assign_type, axis=1)
test.drop(['Flag regul', 'Flag franchise'], axis=1, inplace=True)
test

In [ ]:
#On change le type de la colonne Combi de float à int
test['Combi'] = test['Combi'].astype('Int64')
test = test.iloc[:, -4:]
test

In [ ]:
#Afin d'éviter d'exporter tout le tableau (processus assez long), on n'exporte que les 4 dernières colonnes qu'on pourra copier/coller pour ajouter à la requête (+ ou - 10 secondes)

#############################################################
#############################################################
test.to_excel('../2025/T1/combi 363 delegation type_remb et zpy.xlsx', index=False) #Il faudra changer le chemin d'enregistrement d'un trimestre/année à l'autre
#############################################################
#############################################################